# Libraries

In [1]:
import os
import sys
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")

import shutil

%matplotlib inline

# import cv2
from sklearn.model_selection import train_test_split

from scipy import ndimage

from tqdm import tqdm_notebook #, tnrange
#from itertools import chain
from skimage.io import imread, imshow #, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from skimage import filters

from imgaug import augmenters as iaa

from tqdm import tqdm
from pathlib import Path

import cv2
from sklearn.model_selection import StratifiedKFold
import torch
from torch import nn
from torch import Tensor
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
import torch.backends.cudnn as cudnn
import torch.backends.cudnn
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision.transforms import ToTensor, ToPILImage, Normalize, Compose

import PIL

from datetime import datetime
import json
import gc

import time
#t_start = time.time()

# Global variable

In [2]:
TRAIN_IMG_PATH = "train/images/"
TEST_IMG_PATH = "test/images/"
DEPTH_PATH = "depths.csv"
TRAIN_MASK_PATH = "train/masks/"
TRAIN_INFO_PATH = "train.csv"

# basic parameters
IMG_ORI_SIZE = 101
IMG_TAR_SIZE = 128
SCALE = 1

# Model parameters
START_NEURONS = 16
DROPOUT_RATIO = 0.5

LOAD_CHECKPONT = True

INPUT_CHANNEL = 1

MODEL1_ADAM_LR = 0.01
MODEL1_EPOCHS = 100
MODEL1_BATCH_SIZE = 64
MODEL1_STEPS_PER_EPOCH_TRAIN = 200
MODEL1_LOSS = 'binary_crossentropy'

MODEL2_ADAM_LR = 0.01
MODEL2_EPOCHS = 100
MODEL2_BATCH_SIZE = 64
MODEL2_STEPS_PER_EPOCH_TRAIN = 200
MODEL2_LOSS = 'lovasz_loss'

# ReduceLROnPlateau parameters
MODEL1_REDUCE_FACTOR = 0.5
MODEL1_REDUCE_PATIENT = 5

MODEL2_REDUCE_FACTOR = 0.5
MODEL2_REDUCE_PATIENT = 5

# DICE_BCE_LOSS Parameters
BCE_WEIGHT = 1
DICE_WEIGHT = 0

# Augmentation Parmeters
AUG = True
FIT_METHOD = 'resize_pad'
PAD_METHOD = 'edge'
KFOLD = True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
# encoding=utf8
import cv2
import numpy as np
from torch import nn
from functools import reduce

################################################################################
# related functions & loss functions
################################################################################


def upsample(img):
    if IMG_ORI_SIZE == IMG_TAR_SIZE:
        return img
    return cv2.resize(img, (IMG_TAR_SIZE, IMG_TAR_SIZE))


def downsample(img):
    if IMG_ORI_SIZE == IMG_TAR_SIZE:
        return img
    return cv2.resize(img, (IMG_ORI_SIZE, IMG_ORI_SIZE))


def add_depth_channels(image_array, depth):
    image_array[:,:,1] = depth
    image_array[:,:,2] = image_array[:,:,0] * image_array[:,:,1]
    return image_array


class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

        
def write_event(log, step: int, **data):
    data['step'] = step
    data['dt'] = datetime.now().isoformat()
    log.write(json.dumps(data, sort_keys=True, cls=MyEncoder))
    log.write('\n')
    log.flush()

    
def get_variable(x):
    """ Converts tensors to cuda, if available. """
    if torch.cuda.is_available():
        return x.cuda()
    return x


def get_numpy(x):
    """ Get numpy array for both cuda and not. """
    if torch.cuda.is_available():
        return x.cpu().data.numpy()
    return x.data.numpy()


def get_true_target(targets):
    truth_image = targets.squeeze(3).sum(2).sum(1) > 0
    return truth_image
  

def get_logits_outputs(outputs_image, outputs_pixel):
    batch_size, C, H, W = outputs_pixel.shape
    zero_mask = torch.zeros([batch_size, C, H, W], dtype=torch.float).to(device)
    empty_label = outputs_image<0
    outputs_pixel[empty_label] = zero_mask[empty_label]
    return outputs_pixel


def iou_numpy(outputs, labels):
    SMOOTH = 1e-6
    labels = labels.squeeze(1)
    outputs = outputs.squeeze(1)
    
    intersection = (outputs & labels).sum((1, 2))
    union = (outputs | labels).sum((1, 2))
    
    iou = (intersection + SMOOTH) / (union + SMOOTH)
    
    thresholded = np.ceil(np.clip(20 * (iou - 0.5), 0, 10)) / 10
    
    return thresholded.mean()


def my_iou_metric(label, pred):
    return iou_numpy(pred > 0.5, label>0.5)


def my_iou_metric_2(label, pred):
    return iou_numpy(pred > 0, label>0.5)


def my_iou_metric_pad(label, pred):
    pad_size = (IMG_TAR_SIZE-SCALE*IMG_ORI_SIZE)//2
    return iou_numpy(pred[:,:,pad_size:-pad_size-1,pad_size+1:-pad_size]>0,label[:,:,pad_size:-pad_size-1,pad_size+1:-pad_size]>0.5)

    
def save_checkpoint(state, is_best, filename):
    check_filename = 'checkpoint_{}.pth.tar'.format(filename)
    torch.save(state, check_filename)
    if is_best:
        shutil.copyfile(check_filename, 'model_best_{}.pth.tar'.format(filename))
        

class EarlyStopping(object):
    def __init__(self, mode='min', min_delta=0, patience=10):
        self.mode = mode
        self.min_delta = min_delta
        self.patience = patience
        self.best = None
        self.num_bad_epochs = 0
        self.is_better = None
        self._init_is_better(mode, min_delta)

        if patience == 0:
            self.is_better = lambda a, b: True

    def step(self, metrics):
        if self.best is None:
            self.best = metrics
            return False

        if np.isnan(metrics):
            return True

        if self.is_better(metrics, self.best):
            self.num_bad_epochs = 0
            self.best = metrics
        else:
            self.num_bad_epochs += 1

        if self.num_bad_epochs >= self.patience:
            return True

        return False

    def _init_is_better(self, mode, min_delta):
        if mode not in {'min', 'max'}:
            raise ValueError('mode ' + mode + ' is unknown!')
        if mode == 'min':
            self.is_better = lambda a, best: a < best - min_delta
        if mode == 'max':
            self.is_better = lambda a, best: a > best + min_delta


In [4]:
from __future__ import print_function, division

import torch
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
try:
    from itertools import  ifilterfalse
except ImportError: # py3k
    from itertools import  filterfalse
    

class DiceLoss(nn.Module):
    def __init__(self, smooth=0, eps=1e-7):
        super(DiceLoss, self).__init__()
        self.smooth = smooth
        self.eps = eps

    def forward(self, output, target):
        output = torch.sigmoid(output)
        return 1 - (2 * torch.sum(output * target) + self.smooth) / (torch.sum(output) + torch.sum(target) + self.smooth + self.eps)


class Dice_Bce_Loss(nn.Module):
    def __init__(self, smooth=0, eps=1e-7, dice_weight=0.5, 
                 dice_loss=None, bce_weight=0.9, bce_loss=None):
        super(Dice_Bce_Loss, self).__init__()
        self.smooth = smooth
        self.eps = eps
        self.dice_weight = dice_weight
        self.bce_weight = bce_weight
        self.bce_loss = bce_loss
        self.dice_loss = dice_loss
        
        if self.bce_loss is None:
            self.bce_loss = F.binary_cross_entropy_with_logits
        if self.dice_loss is None:
            self.dice_loss = DiceLoss(smooth, eps)
            
        self.activation = torch.sigmoid
            
    def forward(self, output, target):
        output = self.activation(output)
        return self.dice_weight * self.dice_loss(output, target) + self.bce_weight * self.bce_loss(output, target)


class FocalLoss(nn.Module):
    def __init__(self, alpha=0.5, gamma=2, logits=True, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce

    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss
        

"""
Lovasz-Softmax and Jaccard hinge loss in PyTorch
Maxim Berman 2018 ESAT-PSI KU Leuven (MIT License)
"""
def lovasz_grad(gt_sorted):
    """
    Computes gradient of the Lovasz extension w.r.t sorted errors
    See Alg. 1 in paper
    """
    p = len(gt_sorted)
    gts = gt_sorted.sum()
    intersection = gts - gt_sorted.cumsum(0)
    union = gts + (1 - gt_sorted).cumsum(0)
    jaccard = 1. - intersection / union
    if p > 1: # cover 1-pixel case
        jaccard[1:p] = jaccard[1:p] - jaccard[0:-1]
    return jaccard


# --------------------------- BINARY LOSSES ---------------------------


def lovasz_hinge(logits, labels, per_image=True, ignore=None):
    """
    Binary Lovasz hinge loss
      logits: [B, H, W] Variable, logits at each pixel (between -\infty and +\infty)
      labels: [B, H, W] Tensor, binary ground truth masks (0 or 1)
      per_image: compute the loss per image instead of per batch
      ignore: void class id
    """
    if per_image:
        loss = mean(lovasz_hinge_flat(*flatten_binary_scores(log.unsqueeze(0), lab.unsqueeze(0), ignore))
                          for log, lab in zip(logits, labels))
    else:
        loss = lovasz_hinge_flat(*flatten_binary_scores(logits, labels, ignore))
    return loss


def lovasz_hinge_flat(logits, labels):
    """
    Binary Lovasz hinge loss
      logits: [P] Variable, logits at each prediction (between -\infty and +\infty)
      labels: [P] Tensor, binary ground truth labels (0 or 1)
      ignore: label to ignore
    """
    if len(labels) == 0:
        # only void pixels, the gradients should be 0
        return logits.sum() * 0.
    signs = 2. * labels.float() - 1.
    errors = (1. - logits * signs)
    errors_sorted, perm = torch.sort(errors, dim=0, descending=True)
    perm = perm.data
    gt_sorted = labels[perm]
    grad = lovasz_grad(gt_sorted)
    loss = torch.dot(F.elu(errors_sorted)+1, grad)
    return loss


def flatten_binary_scores(scores, labels, ignore=None):
    """
    Flattens predictions in the batch (binary case)
    Remove labels equal to 'ignore'
    """
    scores = scores.view(-1)
    labels = labels.view(-1)
    if ignore is None:
        return scores, labels
    valid = (labels != ignore)
    vscores = scores[valid]
    vlabels = labels[valid]
    return vscores, vlabels

# --------------------------- HELPER FUNCTIONS ---------------------------

def mean(l, ignore_nan=False, empty=0):
    """
    nanmean compatible with generators.
    """
    l = iter(l)
    if ignore_nan:
        l = ifilterfalse(np.isnan, l)
    try:
        n = 1
        acc = next(l)
    except StopIteration:
        if empty == 'raise':
            raise ValueError('Empty mean')
        return empty
    for n, v in enumerate(l, 2):
        acc += v
    if n == 1:
        return acc
    return acc / n


class BCE_Lovaz_Loss(nn.Module):
    def __init__(self, per_image=True, ignore=None):
        super(BCE_Lovaz_Loss, self).__init__()
        self.per_image = per_image
        self.ignore = ignore
        
    def forward(self, logits, targets):
        loss_1 = lovasz_hinge(logits.squeeze(1), targets.squeeze(1), self.per_image, self.ignore)
        loss_2 = nn.BCEWithLogitsLoss()(logits, targets)
        loss = loss_1 + loss_2
        return loss
    
class Lovaz_Loss(nn.Module):
    def __init__(self, per_image=True, ignore=None):
        super(Lovaz_Loss, self).__init__()
        self.per_image = per_image
        self.ignore = ignore
        
    def forward(self, logits, targets):
        loss = lovasz_hinge(logits.squeeze(1), targets.squeeze(1), self.per_image, self.ignore)
        return loss

class Fuse_Loss(nn.Module):
    def __init__(self, pixel_loss_func=Lovaz_Loss(), weight_image=0.05, weight_pixel=0.5, weight=1):
        super(Fuse_Loss, self).__init__()
        self.image_loss_func = F.cross_entropy
        self.pixel_loss_func = pixel_loss_func
        self.activation = torch.sigmoid
        self.weight_image = weight_image
        self.weight_pixel = weight_pixel
        self.weight = weight
        
    def forward(self, logits, logit_pixel, logit_image, truth_pixel, truth_image):
        pixel_non_empty = logit_pixel[truth_image,:,:,:]
        mask_non_empty = truth_pixel[truth_image,:,:,:]
        loss_pixel = self.pixel_loss_func(pixel_non_empty, mask_non_empty)
        truth_image = truth_image.type(torch.LongTensor).to(device)
        loss_image = self.image_loss_func(logit_image, truth_image, reduction='elementwise_mean')
        loss = self.pixel_loss_func(logits, truth_pixel)
        return self.weight_pixel*loss_pixel + self.weight_image*loss_image + self.weight*loss

In [5]:
import torch
import math
from torch.optim.lr_scheduler import _LRScheduler

class CosineAnnealingLR_with_Restart(_LRScheduler):
    """Set the learning rate of each parameter group using a cosine annealing
    schedule, where :math:`\eta_{max}` is set to the initial lr and
    :math:`T_{cur}` is the number of epochs since the last restart in SGDR:
    .. math::
        \eta_t = \eta_{min} + \frac{1}{2}(\eta_{max} - \eta_{min})(1 +
        \cos(\frac{T_{cur}}{T_{max}}\pi))
    When last_epoch=-1, sets initial lr as lr.
    It has been proposed in
    `SGDR: Stochastic Gradient Descent with Warm Restarts`_. The original pytorch
    implementation only implements the cosine annealing part of SGDR,
    I added my own implementation of the restarts part.
    Args:
        optimizer (Optimizer): Wrapped optimizer.
        T_max (int): Maximum number of iterations.
        T_mult (float): Increase T_max by a factor of T_mult
        eta_min (float): Minimum learning rate. Default: 0.
        last_epoch (int): The index of last epoch. Default: -1.
        model (pytorch model): The model to save.
        out_dir (str): Directory to save snapshots
        take_snapshot (bool): Whether to save snapshots at every restart
    .. _SGDR\: Stochastic Gradient Descent with Warm Restarts:
        https://arxiv.org/abs/1608.03983
    """

    def __init__(self, optimizer, T_max, T_mult, model, out_dir, take_snapshot, eta_min=0, last_epoch=-1):
        self.T_max = T_max
        self.T_mult = T_mult
        self.Te = self.T_max
        self.eta_min = eta_min
        self.current_epoch = last_epoch

        self.model = model
        self.out_dir = out_dir
        self.take_snapshot = take_snapshot

        self.lr_history = []

        super(CosineAnnealingLR_with_Restart, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        new_lrs = [self.eta_min + (base_lr - self.eta_min) *
                   (1 + math.cos(math.pi * self.current_epoch / self.Te)) / 2
                   for base_lr in self.base_lrs]

        self.lr_history.append(new_lrs)
        return new_lrs

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
        self.last_epoch = epoch
        self.current_epoch += 1

        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

        ## restart
        if self.current_epoch == self.Te:
            print("restart at epoch {:03d}".format(self.last_epoch + 1))

            if self.take_snapshot:
                torch.save({
                    'epoch': self.T_max,
                    'state_dict': self.model.state_dict()
                }, self.out_dir + "Weight/" + 'snapshot_e_{:03d}.pth.tar'.format(self.T_max))

            ## reset epochs since the last reset
            self.current_epoch = 0

            ## reset the next goal
            self.Te = int(self.Te * self.T_mult)
            self.T_max = self.T_max + self.Te

In [6]:
import imgaug as ia
from imgaug import augmenters as iaa
import numpy as np

ia.seed(2018)

def _standardize(img):
    return (img - img.map(np.mean)) / img.map(np.std)

affine_seq = iaa.Sequential([
    # General
    iaa.SomeOf((1, 2),
               [iaa.Fliplr(0.5),
                iaa.Noop(),
                ]),
    iaa.Affine(rotate=(-5, 5), mode='reflect'),
    iaa.Crop(px=(0, 10)),
], random_order=True)

intensity_seq = iaa.Sequential([
    #iaa.Invert(0.3),
    iaa.Sometimes(0.3, iaa.ContrastNormalization((0.5, 1.5))),
    iaa.OneOf([
        iaa.Noop(),
        iaa.Sequential([
            iaa.OneOf([
                iaa.Add((-10, 10)),
                iaa.AddElementwise((-10, 10)),
                iaa.Multiply((0.95, 1.05)),
                iaa.MultiplyElementwise((0.95, 1.05)),
            ]),
        ]),
        iaa.OneOf([
            iaa.GaussianBlur(sigma=(0.0, 1.0)),
            iaa.AverageBlur(k=(2, 5)),
            #iaa.MedianBlur(k=(3, 5))
        ])
    ])
], random_order=False)

tta_intensity_seq = iaa.Sequential([
    iaa.Noop()
], random_order=False)

def compute_random_pad(limit=(-4,4)):
    dy  = IMG_TAR_SIZE - IMG_ORI_SIZE*SCALE
    dy0 = dy//2 + np.random.randint(limit[0],limit[1]) # np.random.choice(dy)
    dy1 = dy - dy0
    dx0 = dy//2 + np.random.randint(limit[0],limit[1]) # np.random.choice(dy)
    dx1 = dy - dx0
    return dy0, dx0, dy1, dx1

def resize_pad_seq(pad_size):
    dy0, dx0, dy1, dx1 = compute_random_pad()
    seq = iaa.Sequential([
        affine_seq,
        iaa.Scale({'height': IMG_ORI_SIZE*SCALE, 'width': IMG_ORI_SIZE*SCALE}),
        iaa.Pad(px=(dy0, dx0, dy1, dx1), pad_mode='edge', keep_size=False),
    ], random_order=False)
    return seq

def resize_pad_seq_eval(pad_size):
    seq = iaa.Sequential([
        iaa.Scale({'height': IMG_ORI_SIZE*SCALE, 'width': IMG_ORI_SIZE*SCALE}),
        iaa.Pad(px=(pad_size, pad_size, pad_size+1, pad_size+1), pad_mode='edge', keep_size=False),
    ], random_order=False)
    return seq

def resize_seq():
    seq = iaa.Sequential([
        affine_seq,
        iaa.Scale({'height': IMG_TAR_SIZE, 'width': IMG_TAR_SIZE})
    ], random_order=False)
    return seq

def resize_seq_eval():
    seq = iaa.Sequential([
        iaa.Scale({'height': IMG_TAR_SIZE, 'width': IMG_TAR_SIZE})
    ], random_order=False)
    return seq

In [7]:
# encoding=utf8
import numpy as np
import pandas as pd
from functools import partial
import cv2
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
import torch.backends.cudnn as cudnn
import torch.backends.cudnn
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision.transforms import ToTensor, Normalize, Compose

## convert salt coverage to class
def cov_to_class_1(mask):
    border = 10
    outer = np.zeros((101-2*border, 101-2*border), np.float32)
    outer = cv2.copyMakeBorder(outer, border, border, border, border, borderType = cv2.BORDER_CONSTANT, value = 1)

    cover = (mask>0.5).sum()
    if cover < 8:
        return 0 # empty
    if cover == ((mask*outer) > 0.5).sum():
        return 1 #border
    if np.all(mask==mask[0]):
        return 2 #vertical

    percentage = cover/(101*101)
    if percentage < 0.15:
        return 3
    elif percentage < 0.25:
        return 4
    elif percentage < 0.50:
        return 5
    elif percentage < 0.75:
        return 6
    else:
        return 7
    
def cov_to_class_2(val):    
    for i in range(0, 11):
        if val * 10 <= i :
            return i

## used to load data from data files
class my_DataLoader:
    def __init__(self, train=False, test=False, Kfold=False, test_size=0.2):
        self.test = test
        self.train = train
        self.Kfold = Kfold
        self.test_size = test_size
        self.num_fold = int(1/test_size)
            
        train_df, self.test_df = self._load_depth()
        
        if self.train:
            self._load_image_mask(train_df)
            train_df["coverage"] = train_df.masks.map(np.sum) / pow(IMG_ORI_SIZE, 2)
            train_df["coverage_class"] = train_df.masks.map(cov_to_class_1)
            self.x_train, self.x_valid, self.y_train, self.y_valid = self._get_train_test_split(train_df, self.Kfold, self.num_fold)
            
        if self.test:
            test_df['images'] = self._load_image_test(self.test_df)
            self.x_test = np.array(self.test_df.images.tolist()).reshape(-1, IMG_ORI_SIZE, IMG_ORI_SIZE, 1)

    @staticmethod
    def _load_image_mask(train_df):
        # load image data & mask data
        train_df['images'] = [np.array(cv2.imread(TRAIN_IMG_PATH + "{}.png".format(idx), 0)) for idx in tqdm_notebook(train_df.index)]
        train_df['masks'] = [np.array(cv2.imread(TRAIN_MASK_PATH + "{}.png".format(idx), 0)) for idx in tqdm_notebook(train_df.index)]
        # Normalize image vectors
        #train_df['images'] /= 255
        #train_df['masks'] /= 255
        
    @staticmethod
    def _load_image_test(test_df):
        return [np.array(cv2.imread(TEST_IMG_PATH + "{}.png".format(idx), 0)) for idx in tqdm_notebook(test_df.index)]

    @staticmethod
    def _load_depth():
        train_df = pd.read_csv(TRAIN_INFO_PATH, index_col="id", usecols=[0])
        depths_df = pd.read_csv(DEPTH_PATH, index_col="id")
        depths_df['z'] = depths_df['z'].astype('float')
        train_df = train_df.join(depths_df)
        test_df = depths_df[~depths_df.index.isin(train_df.index)]
        return train_df, test_df

    ## get train & validation split stratified by salt coverage
    @staticmethod
    def _get_train_test_split(train_df, Kfold, num_fold):
        x_train, x_valid, y_train, y_valid = [], [], [], []
        skf = StratifiedKFold(n_splits=num_fold, random_state=1234, shuffle=True)
        for train_index, valid_index in skf.split(train_df.index.values, train_df.coverage_class):
            x_tr = np.array(train_df.images[train_index].tolist()).reshape(-1, IMG_ORI_SIZE, IMG_ORI_SIZE, 1)
            x_tr = np.append(x_tr, [np.fliplr(x) for x in x_tr], axis=0)
            x_train.append(x_tr)
            x_valid.append(np.array(train_df.images[valid_index].tolist()).reshape(-1, IMG_ORI_SIZE, IMG_ORI_SIZE, 1))
            y_tr = np.array(train_df.masks[train_index].tolist()).reshape(-1, IMG_ORI_SIZE, IMG_ORI_SIZE, 1)
            y_tr = np.append(y_tr, [np.fliplr(y) for y in y_tr], axis=0)
            y_train.append(y_tr)
            y_valid.append(np.array(train_df.masks[valid_index].tolist()).reshape(-1, IMG_ORI_SIZE, IMG_ORI_SIZE, 1))
            if not Kfold:
                break
        return x_train, x_valid, y_train, y_valid

    def get_train(self):
        return self.x_train, self.y_train
    
    def get_valid(self):
        return self.x_valid, self.y_valid

    def get_test_x(self):
        return self.x_test

    def get_test_df(self):
        return self.test_df
    
class ShipDataset(Dataset):
    def __init__(self, data, transform=None, mode='train'):
        if mode == 'train' or mode == 'valid':
            self.x = data[0]
            self.y = data[1]
        elif mode == 'test':
            self.data = data
        else:
            raise RuntimeError('MODE_ERROR')
        self.transform = transform
        self.mode = mode
        self.pad_method = PAD_METHOD
        self.pad_size = (IMG_TAR_SIZE-IMG_ORI_SIZE)//2
        
        if FIT_METHOD == 'resize_pad':
            self.aug_func_eval = partial(resize_pad_seq_eval, self.pad_size)
        elif FIT_METHOD == 'resize':
            self.aug_func_eval = resize_seq_eval
            
        if AUG:
            if FIT_METHOD == 'resize_pad':
                self.aug_func = partial(resize_pad_seq, self.pad_size)
            elif FIT_METHOD == 'resize':
                self.aug_func = resize_seq
        
        if INPUT_CHANNEL == 3:
            self.depth = np.tile(np.linspace(0,1,IMG_TAR_SIZE),[IMG_TAR_SIZE,1]).T

    def __len__(self):
        if self.mode == 'train' or self.mode == 'valid':
            return len(self.x)
        elif self.mode == 'test':
            return len(self.data)
        else:
            raise RuntimeError('MODE_ERROR')
               
    def __getitem__(self, idx):
        if self.mode == 'train':
            if AUG:
                resize_seq_det = self.aug_func().to_deterministic()
                new_x_batch = resize_seq_det.augment_image(self.x[idx])
                new_x_batch = intensity_seq.augment_image(new_x_batch)/255
                new_y_batch = resize_seq_det.augment_image(self.y[idx])/255
            else:
                resize_seq_det = self.aug_func_eval().to_deterministic()
                new_x_batch = resize_seq_det.augment_image(self.x[idx])/255
                new_y_batch = resize_seq_det.augment_image(self.y[idx])/255
            if INPUT_CHANNEL == 3:
                new_x_batch = np.tile(new_x_batch,(1,1,3))
                new_x_batch = add_depth_channels(new_x_batch, self.depth)
            return new_x_batch, new_y_batch
        elif self.mode == 'valid':
            resize_seq_det = self.aug_func_eval().to_deterministic()
            new_x_batch = resize_seq_det.augment_image(self.x[idx])/255
            new_y_batch = resize_seq_det.augment_image(self.y[idx])/255
            if INPUT_CHANNEL == 3:
                new_x_batch = np.tile(new_x_batch,(1,1,3))
                new_x_batch = add_depth_channels(new_x_batch, self.depth)
            return new_x_batch, new_y_batch
        elif self.mode == 'test':
            resize_seq_det = self.aug_func_eval()
            test_data = resize_seq_det.augment_image(self.data[idx])/255
            if INPUT_CHANNEL == 3:
                test_data = np.tile(test_data,(1,1,3))
                new_x_batch = add_depth_channels(test_data, self.depth)
            return test_data
        else:
            raise RuntimeError('MODE_ERROR')
            
def make_loader(data, batch_size, num_workers=4, shuffle=False, transform=None, mode='train'):
        return DataLoader(
            dataset=ShipDataset(data, transform=transform, mode=mode),
            shuffle=shuffle,
            num_workers = num_workers,
            batch_size = batch_size,
            pin_memory=torch.cuda.is_available()
        )

In [8]:
dl = my_DataLoader(train=True, Kfold=KFOLD)
x_train, y_train = dl.get_train()
x_valid, y_valid = dl.get_valid()

#Data augmentation
#x_train = np.append(x_train, [np.fliplr(x) for x in x_train], axis=0)
#y_train = np.append(y_train, [np.fliplr(x) for x in y_train], axis=0)

In [11]:
from torch import nn
from torch.nn import functional as F
import torch
from torchvision import models
import torchvision.models.resnet
from torchvision.models.resnet import BasicBlock, Bottleneck
import torch.utils.model_zoo as model_zoo

"""
This script has been taken (and modified) from :
https://github.com/ternaus/TernausNet
@ARTICLE{arXiv:1801.05746,
         author = {V. Iglovikov and A. Shvets},
          title = {TernausNet: U-Net with VGG11 Encoder Pre-Trained on ImageNet for Image Segmentation},
        journal = {ArXiv e-prints},
         eprint = {1801.05746}, 
           year = 2018
        }
"""

class ConvBn2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding):
        super().__init__()
        self.conv = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding),
                                  nn.BatchNorm2d(out_channels),
                                  )

    def forward(self, x):
        return self.conv(x)


class NoOperation(nn.Module):
    def forward(self, x):
        return x

    
class CSE(nn.Module):
    def __init__(self, in_ch, r=2):
        super(CSE, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.linear_1 = nn.Linear(in_ch, in_ch//r)
        self.linear_2 = nn.Linear(in_ch//r, in_ch)

        
    def forward(self, x):
        batch_size, channel_num, _, _ = x.size()
        input_x = x

        x = self.avg_pool(x).view(batch_size, channel_num)
        x = F.relu(self.linear_1(x), inplace=True)
        x = self.linear_2(x)
        x = x.unsqueeze(-1).unsqueeze(-1)
        x = torch.sigmoid(x)

        x = torch.mul(x, input_x)

        return x


class SSE(nn.Module):
    def __init__(self, in_ch):
        super(SSE, self).__init__()
        self.conv = nn.Conv2d(in_ch, 1, kernel_size=1, stride=1, padding=0, bias=False)

    def forward(self, x):
        input_x = x

        x = self.conv(x)
        x = torch.sigmoid(x)

        x = torch.mul(x, input_x)

        return x


class SCSE(nn.Module):
    def __init__(self, in_ch, r=2):
        super(SCSE, self).__init__()

        self.cSE = CSE(in_ch, r)
        self.sSE = SSE(in_ch)

    def forward(self, x):
        cSE = self.cSE(x)
        sSE = self.sSE(x)

        x = torch.add(cSE,sSE)

        return x
    
    
class Decoder(nn.Module):
    def __init__(self, in_channels, middle_channels, out_channels):
        super(Decoder, self).__init__()
        self.conv1 = ConvBn2d(in_channels, middle_channels, kernel_size=3, padding=1)
        self.conv2 = ConvBn2d(middle_channels, out_channels, kernel_size=3, padding=1)
        self.SCSE = SCSE(out_channels)
        
    def forward(self, x, e=None):
        x = F.interpolate(x, scale_factor=2, mode='bilinear', align_corners=False)
        
        if e is not None:
            x = torch.cat([x,e], 1)
            x = F.dropout2d(x, p = 0.50)
            
        x = F.relu(self.conv1(x), inplace=True)
        x = F.relu(self.conv2(x), inplace=True)
        x = self.SCSE(x)

        return x


class UNetResNet34_DS(nn.Module):

    def __init__(self, dropout_2d=0.2, pretrained=True):
        super().__init__()
        self.dropout_2d = dropout_2d

        self.resnet = torchvision.models.resnet34(pretrained=pretrained)

        self.encoder1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=1, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
        self.encoder2 = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2),
            self.resnet.layer1,
        )
        self.encoder3 = self.resnet.layer2
        self.encoder4 = self.resnet.layer3
        self.encoder5 = self.resnet.layer4
        
        self.center = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            )

        self.decoder5 = Decoder(256+512, 512, 64)
        self.decoder4 = Decoder( 64+256, 256, 64)
        self.decoder3 = Decoder( 64+128, 128, 64)
        self.decoder2 = Decoder( 64+ 64,  64, 64)
        self.decoder1 = Decoder( 64+ 64,  32, 64)
        
        self.fuse_pixel = nn.Sequential(
            nn.Conv2d(320, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )
        
        self.logit_pixel  = nn.Sequential(
            nn.Conv2d( 64, 1, kernel_size=1, padding=0),
        )
        
        self.fuse_image = nn.Sequential(
            nn.Conv2d(512, 64, kernel_size=1),
            nn.ReLU(inplace=True),
        )

        self.logit_image = nn.Sequential(
            nn.Linear(64, 2),
        )
        
        self.logit = nn.Sequential(
            nn.Conv2d(64+64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d( 64,  1, kernel_size=1, padding=0),
        )

    def forward(self, x):
        batch_size, C, H, W = x.shape
        mean= [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]
        x = torch.cat([
            (x-mean[2])/std[2],
            (x-mean[1])/std[1],
            (x-mean[0])/std[0],
        ],1)
        """
        if INPUT_CHANNEL == 1:
            x = torch.cat([x,x,x],1)
        """
        
        e1 = self.encoder1(x) #;print('e1', e1.size())
        e2 = self.encoder2(e1)#;print('e2', e2.size())
        e3 = self.encoder3(e2)#;print('e3', e3.size())
        e4 = self.encoder4(e3)#;print('e4', e4.size())
        e5 = self.encoder5(e4)#;print('e5', e5.size())
        
        f = self.center(e5)        #;print('f', f.size())
        
        d5 = self.decoder5(f,e5)   #;print('d5', d5.size())
        d4 = self.decoder4(d5,e4)  #;print('d4', d4.size())
        d3 = self.decoder3(d4,e3)  #;print('d3', d3.size())
        d2 = self.decoder2(d3,e2)  #;print('d2', d2.size())
        d1 = self.decoder1(d2,e1)  #;print('d1', d1.size())
        
        #hyper column
        d = torch.cat((
            d1,
            F.interpolate(d2, scale_factor= 2, mode='bilinear', align_corners=False),
            F.interpolate(d3, scale_factor= 4, mode='bilinear', align_corners=False),
            F.interpolate(d4, scale_factor= 8, mode='bilinear', align_corners=False),
            F.interpolate(d5, scale_factor=16, mode='bilinear', align_corners=False),
        ), 1)
        d = F.dropout2d(d, p=self.dropout_2d)
        fuse_pixel = self.fuse_pixel(d)
        logit_pixel = self.logit_pixel(fuse_pixel)
        
        e = F.adaptive_avg_pool2d(e5, output_size=[1,1])
        e = F.dropout(e, p=self.dropout_2d)
        fuse_image = self.fuse_image(e)
        fuse_image_flatten = fuse_image.view(fuse_image.size(0), -1)
        logit_image = self.logit_image(fuse_image_flatten)
        
        logit = self.logit(torch.cat([
            fuse_pixel, 
            F.interpolate(fuse_image.view(batch_size,-1,1,1,),scale_factor=128, mode='nearest')],1))

        return logit, logit_pixel, logit_image

In [14]:
def validation_phrase0(model, criterion, valid_loader):
    model.eval()
    losses = []
    iou = []
    with torch.no_grad():
        for inputs, targets in valid_loader:
            inputs = inputs.permute(0,3,1,2).type(torch.FloatTensor).to(device)
            truth_image = get_true_target(targets)
            targets = targets.permute(0,3,1,2).type(torch.FloatTensor).to(device)
            outputs, outputs_pixel, outputs_image = model(inputs)
            loss = criterion(outputs, outputs_pixel, outputs_image, targets, truth_image)
            losses += [loss.item()]
            iou += [my_iou_metric_pad(get_numpy(targets), get_numpy(outputs))]

        valid_loss = np.mean(losses)  # type: float

        valid_iou = np.mean(iou)

        metrics = {'val_loss': valid_loss, 'val_iou': valid_iou}
    return metrics

def train_phrase0(model, criterion, train_loader, optimizer, epoch, 
          scheduler, report_each=10, valid_iou=0):
        model.train()
        random.seed()
        scheduler.step(valid_iou)
        lr = optimizer.param_groups[0]['lr']
        print('Current learning rate: {:.4f}'.format(lr))
        losses = []
        ious = []
        tl = train_loader
        
        try:
            mean_loss = 0
            mean_iou = 0
            for i, (inputs, targets) in enumerate(tl):
                inputs = inputs.permute(0,3,1,2).type(torch.FloatTensor).to(device)
                truth_image = get_true_target(targets)
                targets = targets.permute(0,3,1,2).type(torch.FloatTensor).to(device)
                outputs, outputs_pixel, outputs_image = model(inputs)
                loss = criterion(outputs, outputs_pixel, outputs_image, targets, truth_image)
                optimizer.zero_grad()
                batch_size = inputs.size(0)
                loss.backward()
                optimizer.step()
                losses += [loss.item()]
                ious += [my_iou_metric_pad(get_numpy(targets), get_numpy(outputs))]
                mean_loss = np.mean(losses[-report_each:])
                mean_iou = np.mean(ious[-report_each:])

                if i % report_each == 0:
                    print('Epoch: [{0}][{1}/{2}]\t'
                          'Loss {loss:.4f} ({loss_avg:.4f})\t'
                          'IOU {iou:.3f} ({iou_avg:.3f})'.format(
                           epoch, i, len(tl), loss=losses[-1], loss_avg=mean_loss, iou=ious[-1], iou_avg=mean_iou))

            metrics = {'train_loss': mean_loss, 'train_iou': mean_iou}
            return metrics
        
        except KeyboardInterrupt:
            print('Ctrl+C, saving snapshot')
            print('done.')
            return


def validation_phrase1(model, criterion, valid_loader):
    model.eval()
    losses = []
    iou = []
    with torch.no_grad():
        for inputs, targets in valid_loader:
            inputs = inputs.permute(0,3,1,2).type(torch.FloatTensor).to(device)
            truth_image = get_true_target(targets)
            targets = targets.permute(0,3,1,2).type(torch.FloatTensor).to(device)
            outputs, outputs_pixel, outputs_image = model(inputs)
            loss = criterion(outputs, outputs_pixel, outputs_image, targets, truth_image)
            losses += [loss.item()]
            iou += [my_iou_metric_2(get_numpy(targets), get_numpy(outputs))]

        valid_loss = np.mean(losses)  # type: float

        valid_iou = np.mean(iou)

        metrics = {'val_loss': valid_loss, 'val_iou': valid_iou}
    return metrics

def train_phrase1(model, criterion, train_loader, optimizer, epoch, 
          scheduler, report_each=10, valid_iou=0):
        model.train()
        random.seed()
        scheduler.step()
        lr = optimizer.param_groups[0]['lr']
        print('change learning rate into: {:.4f}'.format(lr))
        losses = []
        ious = []
        tl = train_loader
        
        try:
            mean_loss = 0
            mean_iou = 0
            for i, (inputs, targets) in enumerate(tl):
                inputs = inputs.permute(0,3,1,2).type(torch.FloatTensor).to(device)
                truth_image = get_true_target(targets)
                targets = targets.permute(0,3,1,2).type(torch.FloatTensor).to(device)
                outputs, outputs_pixel, outputs_image = model(inputs)
                loss = criterion(outputs, outputs_pixel, outputs_image, targets, truth_image)
                optimizer.zero_grad()
                batch_size = inputs.size(0)
                loss.backward()
                optimizer.step()
                losses += [loss.item()]
                ious += [my_iou_metric_2(get_numpy(targets), get_numpy(outputs))]
                mean_loss = np.mean(losses[-report_each:])
                mean_iou = np.mean(ious[-report_each:])
                
                for param_group in optimizer.param_groups:
                    lr = param_group['lr']

                if i % report_each == 0:
                    print('Epoch: [{0}][{1}/{2}]\t'
                          'Loss {loss:.4f} ({loss_avg:.4f})\t'
                          'IOU {iou:.3f} ({iou_avg:.3f})'.format(
                           epoch, i, len(tl), loss=losses[-1], loss_avg=mean_loss, iou=ious[-1], iou_avg=mean_iou))

            metrics = {'train_loss': mean_loss, 'train_iou': mean_iou}
            return metrics
        
        except KeyboardInterrupt:
            print('Ctrl+C, saving snapshot')
            print('done.')
            return

In [12]:
criterion_phrase0 = Fuse_Loss(pixel_loss_func=Dice_Bce_Loss()).to(device)
criterion_phrase1 = Fuse_Loss(pixel_loss_func=Lovaz_Loss()).to(device)
#early_stop = EarlyStopping(mode='max', min_delta=0, patience=10)

In [12]:
fold = 0
for x_train_f, y_train_f, x_valid_f, y_valid_f in zip(x_train, y_train, x_valid, y_valid):
    train_loader = make_loader((x_train_f, y_train_f), num_workers=0, batch_size=32, shuffle=True)
    valid_loader = make_loader((x_valid_f, y_valid_f), num_workers=0, batch_size=64, mode='valid')

    res_unet = UNetResNet34_DS(dropout_2d=0.5, pretrained=True)
    res_unet = res_unet.to(device)
    
    #model = nn.DataParallel(model, device_ids=None)

    n_epochs = 30
    start_epoch = 0
    report_each = 20
    valid_losses = []
    valid_ious = []
    train_losses = []
    train_ious = []
    valid_iou = 0
    valid_loss = 0
    best_iou = 0
            
    optimizer= SGD(filter(lambda p: p.requires_grad, res_unet.parameters()),
                   0.01, weight_decay=0.0002, momentum=0.9)
    scheduler = ReduceLROnPlateau(optimizer, mode='max', verbose=True, patience=5, factor=0.5, min_lr=0.0001)
        
    print('Fold: [{0}]\t'.format(fold))
    for epoch in range(start_epoch, start_epoch + n_epochs + 1):
        time0 = time.time()
        train_metrics = train_phrase0(res_unet, criterion_phrase0, train_loader, optimizer, 
                              epoch, scheduler, report_each, valid_iou)
        valid_metrics = validation_phrase0(res_unet, criterion_phrase0, valid_loader)

        train_loss = train_metrics['train_loss']
        train_iou = train_metrics['train_iou']
        train_losses += [train_loss]
        train_ious += [train_iou]
        valid_loss = valid_metrics['val_loss']
        valid_iou = valid_metrics['val_iou']
        valid_losses += [valid_loss]
        valid_ious += [valid_iou]
        is_best = best_iou < valid_iou
        best_iou = max(valid_iou, best_iou)
        print('Epoch: [{0}][Validation]\t' 
              'Val_Loss: {val_loss:.5f}\t' 
              'Val_IOU: {val_iou:.5f}\t'
              'Best Val_IOU: {best_iou:.5f}'.format(epoch, 
                                                    val_loss=valid_loss, 
                                                    val_iou=valid_iou, 
                                                    best_iou=best_iou))
        filename = '{name}_{loss}_{phrase}_{fold}_{mode}_{aug}'.format(name='resnet34',
                                                                 loss='bce_dice',
                                                                 phrase='0',
                                                                 fold=fold,
                                                                 mode='iou',
                                                                 aug='resizepad',)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': res_unet.state_dict(),
            'best_iou': best_iou,
            'optimizer': optimizer.state_dict(),
        }, is_best, filename)
        time1 = time.time()
        print('epoch time: ', time1-time0)
    fold += 1

Fold: [4]	
Current learning rate: 0.0100
Epoch: [0][0/201]	Loss 1.6327 (1.6327)	IOU 0.088 (0.088)
Epoch: [0][20/201]	Loss 1.4321 (1.5310)	IOU 0.469 (0.380)
Epoch: [0][40/201]	Loss 1.4405 (1.4412)	IOU 0.562 (0.438)
Epoch: [0][60/201]	Loss 1.4068 (1.4225)	IOU 0.312 (0.402)
Epoch: [0][80/201]	Loss 1.4448 (1.4216)	IOU 0.375 (0.427)
Epoch: [0][100/201]	Loss 1.3599 (1.4093)	IOU 0.375 (0.411)
Epoch: [0][120/201]	Loss 1.3316 (1.3880)	IOU 0.344 (0.445)
Epoch: [0][140/201]	Loss 1.4006 (1.3655)	IOU 0.469 (0.439)
Epoch: [0][160/201]	Loss 1.3353 (1.3884)	IOU 0.375 (0.458)
Epoch: [0][180/201]	Loss 1.3422 (1.3689)	IOU 0.344 (0.452)
Epoch: [0][200/201]	Loss 1.5817 (1.3859)	IOU 0.375 (0.450)
Epoch: [0][Validation]	Val_Loss: 1.36004	Val_IOU: 0.39045	Best Val_IOU: 0.39045
epoch time:  96.751473903656
Current learning rate: 0.0100
Epoch: [1][0/201]	Loss 1.4490 (1.4490)	IOU 0.500 (0.500)
Epoch: [1][20/201]	Loss 1.4215 (1.3581)	IOU 0.562 (0.430)
Epoch: [1][40/201]	Loss 1.3839 (1.3425)	IOU 0.438 (0.422)
Epoc

Epoch: [10][120/201]	Loss 1.2565 (1.1908)	IOU 0.762 (0.762)
Epoch: [10][140/201]	Loss 1.2285 (1.2021)	IOU 0.856 (0.805)
Epoch: [10][160/201]	Loss 1.3878 (1.2466)	IOU 0.744 (0.777)
Epoch: [10][180/201]	Loss 1.1873 (1.2132)	IOU 0.881 (0.779)
Epoch: [10][200/201]	Loss 1.1966 (1.2138)	IOU 0.825 (0.793)
Epoch: [10][Validation]	Val_Loss: 1.21773	Val_IOU: 0.76690	Best Val_IOU: 0.77758
epoch time:  93.86534810066223
Current learning rate: 0.0100
Epoch: [11][0/201]	Loss 1.4025 (1.4025)	IOU 0.738 (0.738)
Epoch: [11][20/201]	Loss 1.2395 (1.2302)	IOU 0.762 (0.770)
Epoch: [11][40/201]	Loss 1.1516 (1.1936)	IOU 0.825 (0.786)
Epoch: [11][60/201]	Loss 1.3477 (1.2282)	IOU 0.681 (0.764)
Epoch: [11][80/201]	Loss 1.1099 (1.1966)	IOU 0.728 (0.778)
Epoch: [11][100/201]	Loss 1.1748 (1.1963)	IOU 0.847 (0.768)
Epoch: [11][120/201]	Loss 1.1903 (1.2087)	IOU 0.819 (0.792)
Epoch: [11][140/201]	Loss 1.0603 (1.2075)	IOU 0.753 (0.810)
Epoch: [11][160/201]	Loss 1.1175 (1.2111)	IOU 0.859 (0.801)
Epoch: [11][180/201]	Los

Epoch: [20][200/201]	Loss 1.5218 (1.2081)	IOU 0.850 (0.826)
Epoch: [20][Validation]	Val_Loss: 1.21625	Val_IOU: 0.77349	Best Val_IOU: 0.79164
epoch time:  94.30641198158264
Current learning rate: 0.0100
Epoch: [21][0/201]	Loss 1.0805 (1.0805)	IOU 0.862 (0.862)
Epoch: [21][20/201]	Loss 1.1458 (1.1880)	IOU 0.869 (0.838)
Epoch: [21][40/201]	Loss 1.1400 (1.1854)	IOU 0.834 (0.812)
Epoch: [21][60/201]	Loss 1.1763 (1.1973)	IOU 0.859 (0.842)
Epoch: [21][80/201]	Loss 1.2400 (1.2143)	IOU 0.803 (0.790)
Epoch: [21][100/201]	Loss 1.1103 (1.2006)	IOU 0.913 (0.800)
Epoch: [21][120/201]	Loss 1.1429 (1.1758)	IOU 0.794 (0.827)
Epoch: [21][140/201]	Loss 1.2068 (1.1894)	IOU 0.812 (0.856)
Epoch: [21][160/201]	Loss 1.2168 (1.2180)	IOU 0.878 (0.805)
Epoch: [21][180/201]	Loss 1.3397 (1.1809)	IOU 0.869 (0.848)
Epoch: [21][200/201]	Loss 1.1916 (1.1969)	IOU 0.625 (0.815)
Epoch: [21][Validation]	Val_Loss: 1.21447	Val_IOU: 0.78489	Best Val_IOU: 0.79164
epoch time:  94.2219889163971
Epoch    22: reducing learning ra

In [ ]:
fold = 0
for x_train_f, y_train_f, x_valid_f, y_valid_f in zip(x_train, y_train, x_valid, y_valid):
    
    if fold < 2:
        fold += 1
        continue
        
    train_loader = make_loader((x_train_f, y_train_f), num_workers=0, batch_size=32, shuffle=True)
    valid_loader = make_loader((x_valid_f, y_valid_f), num_workers=0, batch_size=64, mode='valid')

    res_unet = UNetResNet34_DS(dropout_2d=0.5, pretrained=False)
    res_unet = res_unet.to(device)

    #early_stop = EarlyStopping(mode='max', min_delta=0, patience=10)
    #model = nn.DataParallel(model, device_ids=None)

    n_epochs = 50
    n_cycle = 5
    start_epoch = 0
    report_each = 20
    valid_losses = []
    valid_ious = []
    train_losses = []
    train_ious = []
    valid_iou = 0
    valid_loss = 0
    
    if LOAD_CHECKPONT:
        resume_path = 'model_best_{name}_{loss}_{phrase}_{fold}_{mode}_{aug}.pth.tar'.format(name='resnet34',
                                                                                             loss='bce_dice',
                                                                                             phrase='0',
                                                                                             fold=fold,
                                                                                             mode='iou',
                                                                                             aug='resizepad')
        if os.path.isfile(resume_path):
            print("=> loading checkpoint '{}'".format(resume_path))
            checkpoint = torch.load(resume_path)
            #start_epoch = checkpoint['epoch']
            #best_iou = checkpoint['best_iou']
            res_unet.load_state_dict(checkpoint['state_dict'])
            #optimizer.load_state_dict(checkpoint['optimizer'])
            
    optimizer= SGD(filter(lambda p: p.requires_grad, res_unet.parameters()),
                   0.01, weight_decay=0.0002, momentum=0.9)
    scheduler = CosineAnnealingLR_with_Restart(optimizer,
                                              T_max=n_epochs,
                                              T_mult=1,
                                              model=res_unet,
                                              out_dir='',
                                              take_snapshot=False,
                                              eta_min=1e-3)
        
    for cycle in range(n_cycle):
        best_iou = 0  
        for epoch in range(start_epoch, start_epoch + n_epochs):
            print('Fold/Cycle: [{0}/{1}]\t'.format(fold, cycle))
            time0 = time.time()
            train_metrics = train_phrase1(res_unet, criterion_phrase1, train_loader, optimizer, 
                                  epoch, scheduler, report_each, valid_iou)
            valid_metrics = validation_phrase1(res_unet, criterion_phrase1, valid_loader)

            train_loss = train_metrics['train_loss']
            train_iou = train_metrics['train_iou']
            train_losses += [train_loss]
            train_ious += [train_iou]
            valid_loss = valid_metrics['val_loss']
            valid_iou = valid_metrics['val_iou']
            valid_losses += [valid_loss]
            valid_ious += [valid_iou]
            is_best = best_iou < valid_iou
            best_iou = max(valid_iou, best_iou)
            print('Epoch: [{0}][Validation]\t' 
                  'Val_Loss: {val_loss:.5f}\t' 
                  'Val_IOU: {val_iou:.5f}\t'
                  'Best Val_IOU: {best_iou:.5f}'.format(epoch, 
                                                        val_loss=valid_loss, 
                                                        val_iou=valid_iou, 
                                                        best_iou=best_iou))
            filename = '{name}_{loss}_{phrase}_{fold}_{cycle}_{mode}'.format(name='resnet34',
                                                                     loss='bce_dice',
                                                                     phrase='1',
                                                                     fold=fold,
                                                                     cycle=cycle,
                                                                     mode='iou',)
            save_checkpoint({
                'epoch': epoch + 1,
                'state_dict': res_unet.state_dict(),
                'best_iou': best_iou,
                'optimizer': optimizer.state_dict(),
            }, is_best, filename)
            time1 = time.time()
            print('epoch time: ', time1-time0)
    fold += 1
    """
    if early_stop.step(valid_iou):
        break
    """

=> loading checkpoint 'model_best_resnet34_bce_dice_0_2_iou_resizepad.pth.tar'
Fold/Cycle: [2/0]	
change learning rate into: 0.0100
Epoch: [0][0/200]	Loss 1.2659 (1.2659)	IOU 0.869 (0.869)
Epoch: [0][20/200]	Loss 2.8634 (2.9910)	IOU 0.666 (0.545)
Epoch: [0][40/200]	Loss 2.2464 (2.4884)	IOU 0.656 (0.555)
Epoch: [0][60/200]	Loss 2.1685 (2.1939)	IOU 0.628 (0.634)
Epoch: [0][80/200]	Loss 1.8515 (1.9559)	IOU 0.712 (0.688)
Epoch: [0][100/200]	Loss 2.2182 (2.0788)	IOU 0.488 (0.635)
Epoch: [0][120/200]	Loss 1.2518 (1.8549)	IOU 0.825 (0.694)
Epoch: [0][140/200]	Loss 2.2507 (1.6412)	IOU 0.556 (0.736)
Epoch: [0][160/200]	Loss 2.5296 (1.7292)	IOU 0.672 (0.714)
Epoch: [0][180/200]	Loss 1.6726 (1.7506)	IOU 0.731 (0.710)
Epoch: [0][Validation]	Val_Loss: 1.65892	Val_IOU: 0.73774	Best Val_IOU: 0.73774
epoch time:  106.4466004371643
Fold/Cycle: [2/0]	
change learning rate into: 0.0100
Epoch: [1][0/200]	Loss 2.0718 (2.0718)	IOU 0.597 (0.597)
Epoch: [1][20/200]	Loss 1.5306 (1.5535)	IOU 0.725 (0.747)
Epoch

Epoch: [10][Validation]	Val_Loss: 1.12537	Val_IOU: 0.81529	Best Val_IOU: 0.81529
epoch time:  106.85633444786072
Fold/Cycle: [2/0]	
change learning rate into: 0.0088
Epoch: [11][0/200]	Loss 0.6939 (0.6939)	IOU 0.906 (0.906)
Epoch: [11][20/200]	Loss 1.0837 (0.9997)	IOU 0.806 (0.834)
Epoch: [11][40/200]	Loss 0.7725 (0.9590)	IOU 0.869 (0.850)
Epoch: [11][60/200]	Loss 1.4208 (0.9369)	IOU 0.787 (0.843)
Epoch: [11][80/200]	Loss 0.9319 (1.0471)	IOU 0.884 (0.840)
Epoch: [11][100/200]	Loss 1.0421 (0.9162)	IOU 0.822 (0.864)
Epoch: [11][120/200]	Loss 1.0576 (1.0985)	IOU 0.812 (0.813)
Epoch: [11][140/200]	Loss 1.0266 (1.1072)	IOU 0.844 (0.836)
Epoch: [11][160/200]	Loss 0.8207 (1.0401)	IOU 0.850 (0.820)
Epoch: [11][180/200]	Loss 0.4532 (1.0272)	IOU 0.944 (0.827)
Epoch: [11][Validation]	Val_Loss: 1.23039	Val_IOU: 0.80425	Best Val_IOU: 0.81529
epoch time:  106.11326122283936
Fold/Cycle: [2/0]	
change learning rate into: 0.0086
Epoch: [12][0/200]	Loss 0.7826 (0.7826)	IOU 0.866 (0.866)
Epoch: [12][20/2

Epoch: [21][160/200]	Loss 0.5261 (0.8165)	IOU 0.928 (0.862)
Epoch: [21][180/200]	Loss 0.9924 (0.8321)	IOU 0.831 (0.863)
Epoch: [21][Validation]	Val_Loss: 1.13430	Val_IOU: 0.81423	Best Val_IOU: 0.82331
epoch time:  104.91370606422424
Fold/Cycle: [2/0]	
change learning rate into: 0.0061
Epoch: [22][0/200]	Loss 0.8335 (0.8335)	IOU 0.859 (0.859)
Epoch: [22][20/200]	Loss 0.8928 (0.7288)	IOU 0.847 (0.879)
Epoch: [22][40/200]	Loss 0.5824 (0.8079)	IOU 0.934 (0.870)
Epoch: [22][60/200]	Loss 1.2279 (0.8405)	IOU 0.841 (0.858)
Epoch: [22][80/200]	Loss 0.4209 (0.7336)	IOU 0.947 (0.875)
Epoch: [22][100/200]	Loss 0.4855 (0.8602)	IOU 0.944 (0.857)
Epoch: [22][120/200]	Loss 0.6883 (0.7728)	IOU 0.913 (0.872)
Epoch: [22][140/200]	Loss 0.7685 (0.7824)	IOU 0.853 (0.876)
Epoch: [22][160/200]	Loss 1.2389 (0.8096)	IOU 0.797 (0.865)
Epoch: [22][180/200]	Loss 0.8470 (0.8777)	IOU 0.828 (0.860)
Epoch: [22][Validation]	Val_Loss: 1.09401	Val_IOU: 0.82301	Best Val_IOU: 0.82331
epoch time:  105.90518450737
Fold/Cycle

Epoch: [32][120/200]	Loss 0.8099 (0.6682)	IOU 0.863 (0.888)
Epoch: [32][140/200]	Loss 0.5240 (0.5956)	IOU 0.903 (0.898)
Epoch: [32][160/200]	Loss 0.6540 (0.5715)	IOU 0.875 (0.906)
Epoch: [32][180/200]	Loss 0.5263 (0.5846)	IOU 0.925 (0.901)
Epoch: [32][Validation]	Val_Loss: 1.11326	Val_IOU: 0.83647	Best Val_IOU: 0.83828
epoch time:  104.56745076179504
Fold/Cycle: [2/0]	
change learning rate into: 0.0031
Epoch: [33][0/200]	Loss 0.3107 (0.3107)	IOU 0.953 (0.953)
Epoch: [33][20/200]	Loss 0.8164 (0.6222)	IOU 0.866 (0.896)
Epoch: [33][40/200]	Loss 0.6000 (0.6916)	IOU 0.909 (0.887)
Epoch: [33][60/200]	Loss 0.6083 (0.6228)	IOU 0.878 (0.895)
Epoch: [33][80/200]	Loss 0.5274 (0.5579)	IOU 0.931 (0.907)
Epoch: [33][100/200]	Loss 0.4672 (0.5954)	IOU 0.919 (0.902)
Epoch: [33][120/200]	Loss 0.5191 (0.6382)	IOU 0.916 (0.900)
Epoch: [33][140/200]	Loss 0.5629 (0.5535)	IOU 0.891 (0.901)
Epoch: [33][160/200]	Loss 0.2728 (0.5719)	IOU 0.966 (0.908)
Epoch: [33][180/200]	Loss 0.5970 (0.5464)	IOU 0.897 (0.910)


Epoch: [43][80/200]	Loss 0.4982 (0.4733)	IOU 0.928 (0.923)
Epoch: [43][100/200]	Loss 0.5094 (0.5012)	IOU 0.912 (0.918)
Epoch: [43][120/200]	Loss 0.8229 (0.5270)	IOU 0.806 (0.906)
Epoch: [43][140/200]	Loss 0.3316 (0.4731)	IOU 0.959 (0.927)
Epoch: [43][160/200]	Loss 0.6101 (0.4325)	IOU 0.897 (0.932)
Epoch: [43][180/200]	Loss 0.5533 (0.5580)	IOU 0.894 (0.910)
Epoch: [43][Validation]	Val_Loss: 1.18573	Val_IOU: 0.83383	Best Val_IOU: 0.84032
epoch time:  105.81598472595215
Fold/Cycle: [2/0]	
change learning rate into: 0.0012
Epoch: [44][0/200]	Loss 0.2798 (0.2798)	IOU 0.969 (0.969)
Epoch: [44][20/200]	Loss 0.5033 (0.5594)	IOU 0.906 (0.905)
Epoch: [44][40/200]	Loss 0.5490 (0.4757)	IOU 0.884 (0.920)
Epoch: [44][60/200]	Loss 0.3403 (0.4356)	IOU 0.947 (0.930)
Epoch: [44][80/200]	Loss 0.8651 (0.4920)	IOU 0.853 (0.918)
Epoch: [44][100/200]	Loss 0.4369 (0.4459)	IOU 0.922 (0.928)
Epoch: [44][120/200]	Loss 0.5673 (0.5265)	IOU 0.922 (0.912)
Epoch: [44][140/200]	Loss 0.7140 (0.5297)	IOU 0.859 (0.908)
E

...

Epoch: [47][140/200]	Loss 0.2395 (0.3663)	IOU 0.981 (0.943)
Epoch: [47][160/200]	Loss 0.4268 (0.3654)	IOU 0.922 (0.944)
Epoch: [47][180/200]	Loss 0.1909 (0.3416)	IOU 0.988 (0.950)
Epoch: [47][Validation]	Val_Loss: 1.28457	Val_IOU: 0.84053	Best Val_IOU: 0.84355
epoch time:  105.60870933532715
Fold/Cycle: [2/1]	
change learning rate into: 0.0010
Epoch: [48][0/200]	Loss 0.3075 (0.3075)	IOU 0.969 (0.969)
Epoch: [48][20/200]	Loss 0.5832 (0.4262)	IOU 0.906 (0.931)
Epoch: [48][40/200]	Loss 0.3655 (0.3626)	IOU 0.975 (0.946)
Epoch: [48][60/200]	Loss 0.5616 (0.4041)	IOU 0.925 (0.936)
Epoch: [48][80/200]	Loss 0.4397 (0.3793)	IOU 0.906 (0.941)
Epoch: [48][100/200]	Loss 0.5538 (0.4023)	IOU 0.906 (0.937)
Epoch: [48][120/200]	Loss 0.2522 (0.3998)	IOU 0.972 (0.933)
Epoch: [48][140/200]	Loss 0.4256 (0.3606)	IOU 0.922 (0.943)
Epoch: [48][160/200]	Loss 0.3574 (0.4317)	IOU 0.963 (0.930)
Epoch: [48][180/200]	Loss 0.3996 (0.4273)	IOU 0.931 (0.931)
Epoch: [48][Validation]	Val_Loss: 1.22066	Val_IOU: 0.83957	B

Epoch: [8][120/200]	Loss 0.4679 (0.6631)	IOU 0.931 (0.888)
Epoch: [8][140/200]	Loss 0.7244 (0.6836)	IOU 0.862 (0.886)
Epoch: [8][160/200]	Loss 0.7548 (0.5964)	IOU 0.906 (0.904)
Epoch: [8][180/200]	Loss 0.2446 (0.6138)	IOU 0.972 (0.895)
Epoch: [8][Validation]	Val_Loss: 1.30779	Val_IOU: 0.81891	Best Val_IOU: 0.82703
epoch time:  104.83252453804016
Fold/Cycle: [2/2]	
change learning rate into: 0.0091
Epoch: [9][0/200]	Loss 0.3375 (0.3375)	IOU 0.944 (0.944)
Epoch: [9][20/200]	Loss 0.6794 (0.6074)	IOU 0.891 (0.909)
Epoch: [9][40/200]	Loss 0.2313 (0.6478)	IOU 0.975 (0.898)
Epoch: [9][60/200]	Loss 0.8272 (0.5956)	IOU 0.909 (0.906)
Epoch: [9][80/200]	Loss 0.6440 (0.5754)	IOU 0.894 (0.907)
Epoch: [9][100/200]	Loss 0.6413 (0.6061)	IOU 0.881 (0.898)
Epoch: [9][120/200]	Loss 0.5226 (0.6754)	IOU 0.953 (0.890)
Epoch: [9][140/200]	Loss 0.4812 (0.6434)	IOU 0.944 (0.905)
Epoch: [9][160/200]	Loss 0.9160 (0.6201)	IOU 0.828 (0.898)
Epoch: [9][180/200]	Loss 0.6005 (0.6533)	IOU 0.931 (0.903)
Epoch: [9][Vali

Epoch: [19][80/200]	Loss 0.6999 (0.4720)	IOU 0.841 (0.919)
Epoch: [19][100/200]	Loss 0.7435 (0.5960)	IOU 0.875 (0.904)
Epoch: [19][120/200]	Loss 0.6779 (0.6024)	IOU 0.884 (0.899)
Epoch: [19][140/200]	Loss 0.2971 (0.4556)	IOU 0.975 (0.930)
Epoch: [19][160/200]	Loss 0.4111 (0.4310)	IOU 0.944 (0.937)
Epoch: [19][180/200]	Loss 0.4696 (0.4908)	IOU 0.906 (0.912)
Epoch: [19][Validation]	Val_Loss: 1.29087	Val_IOU: 0.84127	Best Val_IOU: 0.84643
epoch time:  105.40113043785095
Fold/Cycle: [2/2]	
change learning rate into: 0.0066
Epoch: [20][0/200]	Loss 0.4941 (0.4941)	IOU 0.912 (0.912)
Epoch: [20][20/200]	Loss 0.4428 (0.4579)	IOU 0.928 (0.925)
Epoch: [20][40/200]	Loss 0.4272 (0.4785)	IOU 0.938 (0.918)
Epoch: [20][60/200]	Loss 0.3872 (0.4966)	IOU 0.947 (0.925)
Epoch: [20][80/200]	Loss 0.6058 (0.5130)	IOU 0.872 (0.912)
Epoch: [20][100/200]	Loss 0.8734 (0.4211)	IOU 0.872 (0.937)
Epoch: [20][120/200]	Loss 0.7349 (0.4611)	IOU 0.869 (0.928)
Epoch: [20][140/200]	Loss 0.6281 (0.5068)	IOU 0.897 (0.919)
E

Epoch: [30][40/200]	Loss 0.5556 (0.4423)	IOU 0.887 (0.926)
Epoch: [30][60/200]	Loss 0.4312 (0.4208)	IOU 0.928 (0.933)
Epoch: [30][80/200]	Loss 0.3377 (0.4015)	IOU 0.963 (0.935)
Epoch: [30][100/200]	Loss 0.2338 (0.3731)	IOU 0.978 (0.944)
Epoch: [30][120/200]	Loss 0.3686 (0.3791)	IOU 0.931 (0.949)
Epoch: [30][140/200]	Loss 0.3506 (0.3750)	IOU 0.941 (0.940)
Epoch: [30][160/200]	Loss 0.2400 (0.4237)	IOU 0.975 (0.931)
Epoch: [30][180/200]	Loss 0.3611 (0.4244)	IOU 0.947 (0.932)
Epoch: [30][Validation]	Val_Loss: 1.28826	Val_IOU: 0.83105	Best Val_IOU: 0.84643
epoch time:  105.33398056030273
Fold/Cycle: [2/2]	
change learning rate into: 0.0036
Epoch: [31][0/200]	Loss 0.2906 (0.2906)	IOU 0.988 (0.988)
Epoch: [31][20/200]	Loss 0.4510 (0.4072)	IOU 0.909 (0.936)
Epoch: [31][40/200]	Loss 0.4531 (0.4219)	IOU 0.956 (0.932)
Epoch: [31][60/200]	Loss 0.5311 (0.4144)	IOU 0.925 (0.938)
Epoch: [31][80/200]	Loss 0.1813 (0.3682)	IOU 0.991 (0.946)
Epoch: [31][100/200]	Loss 0.5479 (0.4088)	IOU 0.903 (0.932)
Epo

Epoch: [41][0/200]	Loss 0.3551 (0.3551)	IOU 0.947 (0.947)
Epoch: [41][20/200]	Loss 0.2737 (0.3884)	IOU 0.966 (0.944)
Epoch: [41][40/200]	Loss 0.3844 (0.3492)	IOU 0.947 (0.947)
Epoch: [41][60/200]	Loss 0.2436 (0.3916)	IOU 0.984 (0.941)
Epoch: [41][80/200]	Loss 0.4766 (0.3141)	IOU 0.903 (0.958)
Epoch: [41][100/200]	Loss 0.2601 (0.3670)	IOU 0.963 (0.945)
Epoch: [41][120/200]	Loss 0.2136 (0.3119)	IOU 0.969 (0.958)
Epoch: [41][140/200]	Loss 0.1937 (0.3577)	IOU 0.984 (0.945)
Epoch: [41][160/200]	Loss 0.3431 (0.3833)	IOU 0.928 (0.938)
Epoch: [41][180/200]	Loss 0.1363 (0.3227)	IOU 0.997 (0.953)
Epoch: [41][Validation]	Val_Loss: 1.32865	Val_IOU: 0.84089	Best Val_IOU: 0.84643
epoch time:  105.47443318367004
Fold/Cycle: [2/2]	
change learning rate into: 0.0014
Epoch: [42][0/200]	Loss 0.2233 (0.2233)	IOU 0.978 (0.978)
Epoch: [42][20/200]	Loss 0.6786 (0.4025)	IOU 0.903 (0.931)
Epoch: [42][40/200]	Loss 0.3103 (0.3380)	IOU 0.972 (0.951)
Epoch: [42][60/200]	Loss 0.4507 (0.3284)	IOU 0.944 (0.951)
Epoch